In [4]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter
import arrow
import csv


In [5]:

cass.set_riot_api_key("RGAPI-08c1e315-008c-46e2-909f-cedbb30421f5") #Get it from Riot Dev Portal https://developer.riotgames.com/

In [6]:
unpulled_summoners = []
pulled_matches = []
match_player_dict = {} 
matchcounter = 0

In [7]:
start_summoner = Summoner(name="Hubba22", region="EUW")
unpulled_summoners.append(start_summoner.puuid)
# A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.
while unpulled_summoners:

    
    with open("match_playerPUUIDs_timestamp.csv","a") as file:
        for key, value in match_player_dict:
            line = f"{key}, {value}\n"
            file.write(line)

    match_player_dict.clear() 

    with open("fullyMinedSumonerPUUID_Rank.csv","a") as file:
            line = f"{new_puuid},{tier}\n"
            file.write(line)



    with open("unpulled_summoners.csv","w") as file:
        wr = csv.writer(file, delimiter="\n")
        wr.writerow()



    new_puuid = unpulled_summoners[0]
    new_summoner = Summoner(puuid=new_puuid, region="EUW")
    tier = str(new_summoner.league_entries.fives.tier)

    match_history = cass.get_match_history(
        continent=new_summoner.region.continent,
        puuid=new_summoner.puuid,
        begin_index = 0,
        end_index = 100)


    for match in match_history:
        if match.id not in pulled_matches:
            pulled_matches.append(match.id)
            participant_ids = []
            matchcounter += 1
            print(matchcounter, match.creation)        

            

            for p in match.participants:
                #if (not p.is_bot) and p.summoner.exists:
                  if len(participant_ids) >= 10:
                    break
                                   
                  if p.summoner is not None and p.summoner.puuid is not None:
                    if p.summoner.puuid not in unpulled_summoners:
                        unpulled_summoners.append(p.summoner.puuid)


                    participant_ids.append(p.summoner.puuid)
                #entries, rank_last_season, ranks
            timestamp = str(match.creation)
            match_player_dict[match.id] = (participant_ids, timestamp)
        
    unpulled_summoners.remove(new_summoner.puuid)

Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Hubba22


NameError: name 'new_puuid' is not defined

In [5]:
print(len(match_player_dict))

51


In [9]:
for key in match_player_dict:
    print(key, match_player_dict[key])

5846005280 (['b3m8FtWaOV9RTL6EL16KnwyfN8Uvxgrzu0kt1RFZRA0OuFhfzI2WL3XZ2SVC26WG43DWp1sORKLPEg', 'IiC6jy4fxvf3qiA6vPIFjJ57KLgIzydeX1hXy8bseYDwqq5Vr5ZpTQUwqc_zD96ju-fAgaJFIEkmsg', 'bsRmJj-JhGKjPgJIKIDhtKmRmHtw8AEDbtfs4ypwLr65bvzZOiY2PZ4QGAEtnWjeQRsYpBvZ-gPuFA', 'EsiYGe2O7c2WQuyinxKixQXaQ52xdSyMVzdMQ6Uj5cPsgaOVoDPHBsPaFidp0bAsW3GUI1MKjVxGTA', 'GwyWTqkR5hfu1ZwOAL2LXyXwOxd2vFS-5dxoYk1-5qrj-ONF8E1nynscOaVkHb5FEA-4yGOjWDc3AA', 'AsOHEhnYzpvrADPrKvpzhvf_x3GXtNVL5FrY-KFcWGHxX43zI1LjwxkVDKODPQLu1gWF0Z6wP68llg', 'LhLcSIe-tXg6E9I66e0TSI5-yCMo4hIvhC1yQlIZnB86T6I-R1Ern7xrEuKSjiWxULnGfW-TS-uSmg', 'DUi-xPkeFKUDiT0EqfJoPlhs1gXXavbsVIhJO15XqQLUrqvOKChSys7qE0ighnFzjJtFPD-hqbaPmQ', 'SEwkgutzWJG8UZOb0TY4xcRqw1pwa7L7jKMX1GIam1EMKT7w-Fq-ne_ivsAFFGXvRLEXkugqHEn0hA', 'ykODW8V-LFKuYSTMx2HgGKkOc6Mettha9KPChSbPgvkCBY_eS5vOn_2Ppkn3ZRJpaSqdyJGhjY_spg'], '2022-04-28T18:37:23+00:00')
5845927224 (['KxhKe3A7R-wyHlnczR5hY_HaGcVOsEq2KpAQkcYFtUOOcrGQclN9Abbnh3xLpQt7rb8ejQHMNSnNGg', 'LhLcSIe-tXg6E9I66e0TSI5-yCMo4hIvhC1yQlIZn